The script shows why it's IMPRACTICAL to merge Netflix dataset (or any other datasets) with IMDB by name and year to get IMDB IDs.

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')

import os
import pandas as pd
from datasets import NetflixData
from utils import read_cache
from tqdm import tqdm

dropbox_path = os.path.abspath("D:/Dropbox/datasets")
onedrive_path = os.path.abspath("D:/OneDrive - Singapore Management University/datasets")

In [2]:
# IMDB title - id dataset

imdb_basics_path = os.path.join(onedrive_path, "imdb", "title.basics.tsv", "data.tsv")

df_imdb_basics = pd.read_csv(imdb_basics_path, sep='\t')
df_imdb_basics.head()

C:\Users\hongtuonie\AppData\Local\Temp\ipykernel_19008\2113926106.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb_basics = pd.read_csv(imdb_basics_path, sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
# IMDB alias - id dataset

imdb_akas_path = os.path.join(onedrive_path, "imdb", "title.akas.tsv", "data.tsv")

df_imdb_akas = pd.read_csv(imdb_akas_path, sep='\t')
df_imdb_akas.head()

C:\Users\hongtuonie\AppData\Local\Temp\ipykernel_19008\1528601165.py:5: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_imdb_akas = pd.read_csv(imdb_akas_path, sep='\t')


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [4]:
# fix movies with non-numeric starting year

idx = df_imdb_basics['startYear'].str.isnumeric() == False
df_imdb_basics['startYear'][idx] = '1700'
print(sum(idx))
print(sum(df_imdb_basics['startYear'] == '\\N'))

1352838

In [7]:
# drop movies after 2006

print(len(df_imdb_basics.index))

idx = df_imdb_basics[  (df_imdb_basics['startYear'].astype(int) >= 2006)  ].index
df_imdb_basics.drop(idx , inplace=True)

print(len(df_imdb_basics.index))

10080285
10080285


In [8]:
# Netflix titles

cache_path = "D:/OneDrive - Singapore Management University/cache/netflix_data_small.pickle"

# cache_path = "D:/OneDrive - Singapore Management University/cache/netflix_data_full.pickle"

X, factor_info = read_cache(cache_path)

print(factor_info[1])

df_netflix = pd.DataFrame()
df_netflix['order'] = factor_info[1][0]
df_netflix['idmap'] = factor_info[1][1]
df_netflix['alias'] = factor_info[1][2]

df_netflix.head()

(array([   0,    1,    2, ..., 3359, 3360, 3361]), array([    3,     9,    17, ..., 17753, 17762, 17765]), array(['Character', "Class of Nuke 'Em High 2", '7 Seconds', ...,
       'Maslin Beach', 'Gattaca', "Godzilla's Revenge"], dtype='<U70'))


,order,idmap,alias
0,0,3,Character
1,1,9,Class of Nuke 'Em High 2
2,2,17,7 Seconds
3,3,19,By Dawn's Early Light
4,4,20,Seeta Aur Geeta


In [9]:
# idx = df_netflix[0].isin(df_imdb_basics['primaryTitle'])
# print(sum(idx))

# idx = df_netflix[0].isin(df_imdb_basics['originalTitle'])
# print(sum(idx))

# idx = df_netflix[0].isin(df_imdb_akas['title'])
# print(sum(idx))

# small
# 2791
# 2594
# 2934

# full
# 9882
# 9148
# 10453

In [10]:
# get movie id

# get year from imdb_basics
df_imdb_akas_year = df_imdb_akas.merge(df_imdb_basics[['tconst', 'startYear']], left_on='titleId', right_on='tconst', how='left')
df_imdb_akas_year.drop(['tconst'], axis=1, inplace=True)
df_imdb_akas_year.head(10)

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle,startYear
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0,1894
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0,1894
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0,1894
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0,1894
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0,1894
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0,1894
6,tt0000001,7,Carmencita,\N,\N,original,\N,1,1894
7,tt0000001,8,カルメンチータ,JP,ja,imdbDisplay,\N,0,1894
8,tt0000002,1,Le clown et ses chiens,\N,\N,original,\N,1,1892
9,tt0000002,2,Le clown et ses chiens,FR,\N,imdbDisplay,\N,0,1892


In [11]:
# get year from movie_titles
titles_path = f'D:\\OneDrive - Singapore Management University\\datasets\\netflix-cornac\\movie_titles.csv'
df_titles = pd.read_csv(titles_path, header=None, names=['item', 'year', 'name'])
df_titles.head(5)
df_netflix_year = df_netflix.merge(df_titles[['year', 'name']], left_on='alias', right_on='name', how='left')
df_netflix_year = df_netflix_year.drop(['name'], axis=1)
df_netflix_year.head(10)

,order,idmap,alias,year
0,0,3,Character,1997
1,1,9,Class of Nuke 'Em High 2,1991
2,2,17,7 Seconds,2005
3,3,19,By Dawn's Early Light,2000
4,3,19,By Dawn's Early Light,1990
5,4,20,Seeta Aur Geeta,1972
6,5,46,Rudolph the Red-Nosed Reindeer,1964
7,6,57,Richard III,1995
8,6,57,Richard III,1955
9,7,58,Dragonheart,1996


In [12]:
# get titleId from df_imdb_akas
df_netflix_imdb_id = df_netflix_year.merge(df_imdb_akas_year[['titleId', 'ordering', 'title', 'isOriginalTitle', 'startYear']], left_on='alias', right_on='title', how='left')
df_netflix_imdb_id.head(10)

,order,idmap,alias,year,titleId,ordering,title,isOriginalTitle,startYear
0,0,3,Character,1997,tt0119448,13.0,Character,0,1997
1,0,3,Character,1997,tt0119448,17.0,Character,0,1997
2,0,3,Character,1997,tt0119448,19.0,Character,0,1997
3,0,3,Character,1997,tt0119448,24.0,Character,0,1997
4,0,3,Character,1997,tt0119448,6.0,Character,0,1997
5,0,3,Character,1997,tt11107306,2.0,Character,1,2019
6,0,3,Character,1997,tt11107306,3.0,Character,0,2019
7,0,3,Character,1997,tt11422690,1.0,Character,1,2020
8,0,3,Character,1997,tt11422690,2.0,Character,0,2020
9,0,3,Character,1997,tt13328316,1.0,Character,0,2020


In [13]:
df_netflix_imdb_id = df_netflix_imdb_id.drop_duplicates(subset=['alias', 'year', 'titleId', 'startYear'])
df_netflix_imdb_id.head(50)

,order,idmap,alias,year,titleId,ordering,title,isOriginalTitle,startYear
0,0,3,Character,1997,tt0119448,13.0,Character,0,1997
5,0,3,Character,1997,tt11107306,2.0,Character,1,2019
7,0,3,Character,1997,tt11422690,1.0,Character,1,2020
9,0,3,Character,1997,tt13328316,1.0,Character,0,2020
10,0,3,Character,1997,tt13569902,4.0,Character,0,2020
11,0,3,Character,1997,tt14118266,10.0,Character,0,2021
25,0,3,Character,1997,tt15302486,1.0,Character,0,2021
26,0,3,Character,1997,tt16748858,1.0,Character,1,2021
28,0,3,Character,1997,tt2245354,1.0,Character,1,2012
31,0,3,Character,1997,tt2989400,1.0,Character,1,2013


In [30]:
df_netflix_imdb_id_same_year = df_netflix_imdb_id[  df_netflix_imdb_id['year'].astype(str) == df_netflix_imdb_id['startYear'].astype(str)  ]
df_netflix_imdb_id_same_year

,order,idmap,alias,year,titleId,ordering,title,isOriginalTitle,startYear
0,0,3,Character,1997,tt0119448,13.0,Character,0,1997
49,2,17,7 Seconds,2005,tt0417395,11.0,7 Seconds,1,2005
71,3,19,By Dawn's Early Light,1990,tt0099197,1.0,By Dawn's Early Light,0,1990
78,4,20,Seeta Aur Geeta,1972,tt0078222,11.0,Seeta Aur Geeta,1,1972
85,5,46,Rudolph the Red-Nosed Reindeer,1964,tt0058536,12.0,Rudolph the Red-Nosed Reindeer,1,1964
...,...,...,...,...,...,...,...,...,...
93059,3357,17742,Catherine the Great,1995,tt0112031,12.0,Catherine the Great,0,1995
93098,3358,17747,Eric Clapton: 24 Nights,1991,tt0191992,1.0,Eric Clapton: 24 Nights,1,1991
93100,3359,17753,Maslin Beach,1997,tt0229564,1.0,Maslin Beach,0,1997
93105,3360,17762,Gattaca,1997,tt0119177,11.0,Gattaca,0,1997
